# Using the heparchy database API

## Writing a new database

In [1]:
import numpy as np

from heparchy.write import DataWriter

Reference dataset taken from https://zenodo.org/record/3981290#.YIgTCi9Q3xW.

First 3 events from wboson.txt have been included.

Description:
* 13 TeV collision data simulated with pythia 8.183.
* wboson.txt contains events generated from a W' boson with a mass of 600 GeV, which decays 100% of the time to a W boson and a Z boson. The W boson is forced to decay haronically and the Z boson decays into neutrinos.
* events in the text format
* each line in the text represent one event, which contains variable number of detector-stable particles.
* each particle contains 7 features in order: [px, py, pz, E, pdgID, is-from-W, is-in-leading-jet]. The first four features are the four momentum of the particle, and pdgID is the pag number of the particle. is-from-W is 1 if the particle coming from W boson and 0 otherwise. is-in-leading-jet is 1 if the particle is inside the leading jet reconstructed from the anti-kT jet algorithm (R=1.0)

In [2]:
with DataWriter('events.hdf5') as f_out: # create and open the database
    with f_out.new_process('wboson') as process: # a new process to contain events
        # --- metadata to describe the process (retrievable when reading) --- #
        process.decay(in_pcls=(2212, 2212), out_pcls=(23, 24))
        process.signal_id(signal_pcl=24)
        process.com_energy(energy=13.0, unit='TeV')
        # ------ #
        with open('reference_data.txt', 'r') as f_in:
            # --- your own code to format the data into numpy arrays --- #
            for evt_num, line in enumerate(f_in): # each text line is an event
                data = np.fromstring(line, sep=' ') # flattened data for event
                num_cols = 7 # specified in description
                num_pcls = len(data) // num_cols # num particles in each column
                data = data.reshape((num_pcls, num_cols)) # reshape flattened array => columns
            # ------ #
                with process.new_event() as event:
                    # add the event data:
                    event.pmu(data[:, :4])
                    event.pdg(data[:, 4])
                    event.is_signal(data[:, 5].astype(np.bool_))
                    event.custom_dataset(
                        name='in_leading',
                        data=data[:, 6].astype(np.bool_),
                        dtype='<?', # little Endian boolean
                        )

## Reading from an existing database

In [3]:
from heparchy.read import EventLoader

In [4]:
with EventLoader('events.hdf5', 'wboson') as evts:
    # --- metadata available before loading event --- #
    num_evts = len(evts)
    in_pcls = evts.get_ue_pcls('in_pcls')
    out_pcls = evts.get_ue_pcls('out_pcls')
    signal_pcl = evts.get_ue_pcls('signal_pcl')
    com_energy = evts.get_com()
    energy_unit = evts.get_unit()
    print(f'Incoming particles with pdgids of {in_pcls} '
          + f'collide with energy of {com_energy} {energy_unit} '
          + f'to produce outgoing particles with pdgids of {out_pcls}. '
          + f'The outgoing particle with pdgid {signal_pcl} is marked as signal.'
         )
    # --- loading the event and extracting datasets --- #
    evt_idx = 0
    evts.set_evt(evt_idx)
    num_pcls = evts.get_num_pcls()
    pmu = evts.get_pmu()
    is_signal = evts.get_signal()
    pdg = evts.get_pdg()
    in_leading = evts.get_custom('in_leading')

Incoming particles with pdgids of [2212, 2212] collide with energy of 13.0 TeV to produce outgoing particles with pdgids of [23, 24]. The outgoing particle with pdgid 24 is marked as signal.


In [5]:
print(f'event {evt_idx} contains {num_pcls} final state particles')

event 0 contains 255 final state particles


In [6]:
print(pmu)

[[-2.56342e-01  5.58407e-01 -8.28669e-01  1.04101e+00]
 [-5.90604e-02 -9.99182e-02 -1.15661e+00  1.26291e+00]
 [ 3.45896e-01 -2.91281e-01 -6.17557e+01  6.17576e+01]
 ...
 [ 2.92506e-02  1.19800e-01  2.69329e-01  2.96220e-01]
 [-1.99885e-02  1.33609e-01  1.97177e-01  2.39018e-01]
 [ 2.31011e-02  8.24350e-02  4.18310e-03  8.57128e-02]]


In [7]:
print(is_signal)

[False False False False False False False False False False False False
 False False False False  True  True  True  True  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True  True  True  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [8]:
print(pdg)

[ -211   321   211   211   211 -2212   211  -321   211   321  -211  -211
  -321   321  -211  -211  -211   211   211  -211 -2112  -211  -321   211
  -211   321   211  -211  -211  2212   321  -211   321  -321   211  2212
 -2112  -211  2212 -2212   211  2112 -2112  -211  2212   211  -211   321
  -211  -211  -211  2112   211   211  -211  -211   211   211   211   211
  -211  -211  -321   211   211   211  -211  -211   211  2112   211  -211
   211  -211   211  -211   130  -321   211   211  -211  -211   211  -211
  -211   211   211  -211   130   130    22    22   211  -211   211  -211
   211  -211    22    22    22    22   211  -211    22    22   211  -211
   211  -211   211  -211    22    22    22   211  -211    22    22    22
    22    22    22    22    22   211  -211  2212  -211 -2112   211   211
  -211    22    22    22    22    22    22    22    22   211  -211   211
  -211    22    22 -2212   211  2112    22    22   211  -211    22    22
    22    22    22    22    22    22    22    22   

In [9]:
print(in_leading)

[False False False False False False False False False False False False
 False False False False  True  True False False  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True  True False  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

### Looping through events of a given process
The object returned by `EventLoader` is actually an iterable,
so you need not set the event numbers explicitly.

In [10]:
with EventLoader('events.hdf5', 'wboson') as evts:
    for evt_idx, evt in enumerate(evts):
        print(f'Event {evt_idx} has {evt.get_num_pcls()} final state particles')

Event 0 has 255 final state particles
Event 1 has 700 final state particles
Event 2 has 230 final state particles
